In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from itertools import combinations

In [2]:
data = pd.read_csv('movie_bd_v5.csv')

# Предобработка

In [3]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...

data['profit'] = data.revenue - data.budget
data_ini = data

# 1. У какого фильма из списка самый большой бюджет?

In [4]:
# тут пишем ваш код для решения данного вопроса:
data[data.budget == max(data.budget)][['imdb_id', 'original_title', 'budget']]

,imdb_id,original_title,budget
723,tt1298650,Pirates of the Caribbean: On Stranger Tides,380000000


In [5]:
answers['1'] = '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)'

# 2. Какой из фильмов самый длительный (в минутах)?

In [6]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = '2. Gods and Generals (tt0279111)'

In [7]:
data[data.runtime == max(data.runtime)][['original_title', 'imdb_id', 'runtime']]

,original_title,imdb_id,runtime
1157,Gods and Generals,tt0279111,214


# 3. Какой из фильмов самый короткий (в минутах)?





In [8]:
data[data.runtime == min(data.runtime)][['original_title', 'imdb_id', 'runtime']]

,original_title,imdb_id,runtime
768,Winnie the Pooh,tt1449283,63


In [9]:
answers['3'] = '3. Winnie the Pooh (tt1449283)'

# 4. Какова средняя длительность фильмов?


In [10]:
data.runtime.mean()

109.6585494970884

In [11]:
answers['4'] = '2. 110'

# 5. Каково медианное значение длительности фильмов? 

In [12]:
data.runtime.median()

107.0

In [13]:
answers['5'] = '1. 107'

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [14]:
# лучше код получения столбца profit вынести в Предобработку что в начале
data[data.profit == max(data.profit)][['original_title', 'imdb_id', 'profit']]

,original_title,imdb_id,profit
239,Avatar,tt0499549,2544505847


In [15]:
answers['6'] = '5. Avatar (tt0499549)'

# 7. Какой фильм самый убыточный? 

In [16]:
data[data.profit == min(data.profit)][['original_title', 'imdb_id', 'profit']]

,original_title,imdb_id,profit
1245,The Lone Ranger,tt1210819,-165710090


In [17]:
answers['7'] = '4. The Lone Ranger (tt1210819)'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [18]:
len(data[data.profit > 0])

1478

In [19]:
answers['8'] = '1. 1478'

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [20]:
list2008 = data[data.release_year == 2008]
list2008[list2008.revenue == max(list2008.revenue)][
    ['original_title', 'imdb_id', 'release_year', 'revenue']]

,original_title,imdb_id,release_year,revenue
599,The Dark Knight,tt0468569,2008,1001921825


In [21]:
answers['9'] = '4. The Dark Knight (tt0468569)'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [22]:
worst = data[data.release_year.isin(range(2012, 2015))][
    ['original_title', 'imdb_id', 'release_year', 'profit']]
worst[worst.profit == min(worst.profit)]

,original_title,imdb_id,release_year,profit
1245,The Lone Ranger,tt1210819,2013,-165710090


In [23]:
answers['10'] = '5. The Lone Ranger (tt1210819)'

# 11. Какого жанра фильмов больше всего?

In [24]:
data = data_ini
pd.Series(data.genres.str.cat(sep='|').split('|')).value_counts().head()

Drama        782
Comedy       683
Thriller     596
Action       582
Adventure    415
dtype: int64

In [25]:
answers['11'] = '3. Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [26]:
data = data_ini
genres = pd.Series(data[data.profit > 0].genres.str.cat(sep='|').split('|'))
genres.value_counts().head()

Drama        560
Comedy       551
Thriller     446
Action       444
Adventure    337
dtype: int64

In [27]:
answers['12'] = '1. Drama'

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [28]:
# variant 1
from heapq import nlargest
directors = set(data.director.str.cat(sep='|').split('|'))
revenues = {director: sum(data[data.director.str.contains(director)].revenue) 
            for director in directors}
nlargest(3, revenues, key=revenues.get)

['Peter Jackson', 'Christopher Nolan', 'David Yates']

In [29]:
# variant 2
from operator import itemgetter
directors = set(data.director.str.cat(sep='|').split('|'))
revenues = {director: sum(data[data.director.str.contains(director)].revenue) 
            for director in directors}
sorted(revenues.items(), key=itemgetter(1), reverse=True)[:3]

[('Peter Jackson', 6490593685),
 ('Christopher Nolan', 4167548502),
 ('David Yates', 4154295625)]

In [30]:
# variant 3, the best, does not require any extra libraries
directors = set(data.director.str.cat(sep='|').split('|'))
revenues = {director: sum(data[data.director.str.contains(director)].revenue) 
            for director in directors}
pd.DataFrame.from_dict(revenues, orient='index', columns=['revenue']).nlargest(3, 'revenue')

,revenue
Peter Jackson,6490593685
Christopher Nolan,4167548502
David Yates,4154295625


In [31]:
answers['13'] = '5. Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [32]:
# variant 1
data = data_ini
data = data[data.genres.map(lambda x: 'Action' in x)][['director']]
directors = data.director.str.cat(sep='|').split('|')
pd.Series(directors).value_counts().head()

Robert Rodriguez      9
Michael Bay           7
Paul W.S. Anderson    7
Antoine Fuqua         6
Ridley Scott          6
dtype: int64

In [33]:
# variant 2
data = data_ini
data = data[data.genres.str.contains('Action')][['director']]
directors = data.director.str.cat(sep='|').split('|')
pd.Series(directors).value_counts().head()

Robert Rodriguez      9
Michael Bay           7
Paul W.S. Anderson    7
Antoine Fuqua         6
Ridley Scott          6
dtype: int64

In [34]:
answers['14'] = '3. Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [35]:
data = data_ini
data_2012 = data[data.release_year == 2012]
cast = set(data_2012.cast.str.cat(sep='|').split('|'))
pd.Series({x: sum(data_2012[data_2012.cast.str.contains(x)].revenue) for x in cast}).nlargest()

Chris Hemsworth       2027450773
Denis Leary           1629460639
Anne Hathaway         1522851057
Chris Evans           1519557910
Scarlett Johansson    1519557910
dtype: int64

In [36]:
answers['15'] = '3. Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [37]:
data = data_ini
pd.Series(data[data.budget > data.budget.mean()].cast.str.cat(sep='|').split('|')).value_counts()

Matt Damon           18
Adam Sandler         17
Angelina Jolie       16
Samuel L. Jackson    15
Eddie Murphy         15
                     ..
Rose Leslie           1
Johnny Strong         1
Jesse Bradford        1
Nick Chinlund         1
Willow Smith          1
Length: 1505, dtype: int64

In [38]:
answers['16'] = '3. Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [39]:
data = data_ini
genres = data[data.cast.str.contains('Nicolas Cage')].genres
pd.Series(genres.str.cat(sep='|').split('|')).value_counts()

Action             17
Thriller           15
Drama              12
Crime              10
Fantasy             8
Adventure           7
Comedy              6
Science Fiction     4
Family              3
Mystery             3
Animation           3
History             2
War                 1
Romance             1
Horror              1
dtype: int64

In [40]:
answers['17'] = '3. Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [41]:
data = data_ini
Paramount = data[data.production_companies.str.contains('Paramount Pictures')]
Paramount[Paramount.profit == min(Paramount.profit)][['original_title', 'imdb_id']]

,original_title,imdb_id
925,K-19: The Widowmaker,tt0267626


In [42]:
answers['18'] = '1. K-19: The Widowmaker (tt0267626)'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [43]:
data = data_ini
data.groupby('release_year').sum().sort_values('revenue', ascending=False).revenue.head()

release_year
2015    25449202382
2014    23405862953
2013    23213799791
2012    23079001687
2011    22676791872
Name: revenue, dtype: int64

In [44]:
answers['19'] = '5. 2015'

# 20. Какой самый прибыльный год для студии Warner Bros?

In [45]:
data = data_ini
Warner = data[data.production_companies.str.contains('Warner Bros')][['release_year', 'profit']]
Warner.groupby('release_year').sum().nlargest(3, columns='profit')

,profit
release_year,
2014,2295464519
2007,2201675217
2008,2134595031


In [46]:
answers['20'] = '1. 2014'

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [47]:
data = data_ini.release_date
# data
# datetime.strptime(data[0], '%m/%d/%Y').month
months = data.apply(lambda x: datetime.strptime(x, '%m/%d/%Y').month)
months.value_counts().sort_values(ascending=False)

9     227
12    190
10    186
8     161
3     156
4     149
6     147
11    146
7     142
5     140
2     135
1     110
Name: release_date, dtype: int64

In [48]:
answers['21'] = '4. сентябрь'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [49]:
data = data_ini.release_date
months = data.apply(lambda x: datetime.strptime(x, '%m/%d/%Y').month)
len(months[(months > 5) & (months < 9)])

450

In [50]:
answers['22'] = '2. 450'

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [51]:
# who released most in winter
data = data_ini[['director', 'release_date']]
data['release_month'] = data.release_date.apply(lambda x: x.split('/')[0])
data = data[data.release_month.isin(['12', '1', '2'])]
pd.Series(data.director.str.cat(sep='|').split('|')).value_counts()

C:\Users\kan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Peter Jackson         7
Steven Soderbergh     6
Clint Eastwood        6
Nancy Meyers          4
Martin Scorsese       4
                     ..
Tony Leech            1
Charles Stone III     1
Dewey Nicks           1
GrÃ©gory Levasseur    1
Chris Weitz           1
Length: 358, dtype: int64

In [52]:
answers['23'] = '5. Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [53]:
data = data_ini[['production_companies', 'original_title']]
data['title_len'] = data.original_title.apply(len)
# data.head()
studios = set(data.production_companies.str.cat(sep='|').split('|'))
# for studio in studios:
#     print(studio)
pd.Series([data[data.production_companies.str.contains(studio)]['title_len'].mean()
           for studio in studios], index=studios).sort_values(ascending=False)

C:\Users\kan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\kan\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\strings.py:1843: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


Four By Two Productions                   83.0
Jim Henson Company, The                   59.0
Dos Corazones                             47.0
Polsky Films                              46.0
Museum Canada Productions                 46.0
                                          ... 
Picture Perfect (I)                        NaN
World Wrestling Entertainment (WWE)        NaN
British Broadcasting Corporation (BBC)     NaN
Palomar Pictures (II)                      NaN
Edmonds Entertainment Group (EEG)          NaN
Length: 1771, dtype: float64

In [54]:
answers['24'] = '5. Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [55]:
# variant 1
data = data_ini[['production_companies', 'overview']]
data['length'] = data.overview.apply(len)
studios = set(data.production_companies.str.cat(sep='|').split('|'))
mean_overview = pd.DataFrame(columns=['length'])
for studio in studios:
    mean_overview.loc[studio] = data[data.production_companies.str.contains(studio)].length.mean()
mean_overview.sort_values('length', ascending=False).head(3)

C:\Users\kan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,length
Midnight Picture Show,1000.0
Room 9 Entertainment,964.0
Brookwell-McNamara Entertainment,936.0


In [56]:
# variant 2
data = data_ini[['production_companies', 'overview']]
data['length'] = data.overview.apply(len)
studios = set(data.production_companies.str.cat(sep='|').split('|'))
mean_overview = pd.DataFrame(columns=['length'])
for studio in studios:
    mean_overview.loc[studio] = data[data.production_companies.str.contains(studio)].length.mean()
mean_overview.nlargest(3, 'length')

C:\Users\kan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,length
Midnight Picture Show,1000.0
Room 9 Entertainment,964.0
Brookwell-McNamara Entertainment,936.0


In [57]:
# variant 3
data = data_ini[['production_companies', 'overview']]
data['length'] = data.overview.apply(len)
studios = set(data.production_companies.str.cat(sep='|').split('|'))
mean_overview = pd.DataFrame(columns=['length'])
for studio in studios:
    mean_overview.loc[studio] = data[data.production_companies.str.contains(studio)].length.mean()
mean_overview.index[mean_overview.length == max(mean_overview.length)].tolist()

C:\Users\kan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


['Midnight Picture Show']

In [58]:
answers['25'] = '3. Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [59]:
data = data_ini[['original_title', 'vote_average']]
data.nlargest(int(len(data)/100), 'vote_average').sort_values('original_title')

,original_title,vote_average
1191,12 Years a Slave,7.9
124,Big Hero 6,7.8
128,Gone Girl,7.9
119,Guardians of the Galaxy,7.9
370,Inception,7.9
9,Inside Out,8.0
118,Interstellar,8.0
1800,Memento,7.9
34,Room,8.0
28,Spotlight,7.8


In [60]:
answers['26'] = '1. Inside Out, The Dark Knight, 12 Years a Slave'

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [61]:
# variant 1
data = data_ini[['cast']]
frequency = pd.Series()

for index, row in data.iterrows():
    cast = sorted(row[0].split('|'))
    pairs = ['|'.join(pair) for pair in combinations(cast, 2)]
    for pair in pairs:
        try:
            frequency[pair] += 1
        except:
            frequency[pair] = 1

frequency.sort_values(ascending=False)[:5]

Emma Watson|Rupert Grint            8
Daniel Radcliffe|Rupert Grint       8
Daniel Radcliffe|Emma Watson        8
Helena Bonham Carter|Johnny Depp    6
Ben Stiller|Owen Wilson             6
dtype: int64

In [62]:
# variant 2
data = data_ini[['cast']]
frequency = pd.Series()

for index, row in data.iterrows():
    cast = sorted(row[0].split('|'))
    pairs = ['|'.join(pair) for pair in combinations(cast, 2)]
    for pair in pairs:
        if pair in frequency.index:
            frequency[pair] += 1
        else:
            frequency[pair] = 1

frequency.sort_values(ascending=False)[:5]

Emma Watson|Rupert Grint            8
Daniel Radcliffe|Rupert Grint       8
Daniel Radcliffe|Emma Watson        8
Helena Bonham Carter|Johnny Depp    6
Ben Stiller|Owen Wilson             6
dtype: int64

In [63]:
answers['27'] = '5. Daniel Radcliffe & Rupert Grint'

# Submission

In [64]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '2. Gods and Generals (tt0279111)',
 '3': '3. Winnie the Pooh (tt1449283)',
 '4': '2. 110',
 '5': '1. 107',
 '6': '5. Avatar (tt0499549)',
 '7': '4. The Lone Ranger (tt1210819)',
 '8': '1. 1478',
 '9': '4. The Dark Knight (tt0468569)',
 '10': '5. The Lone Ranger (tt1210819)',
 '11': '3. Drama',
 '12': '1. Drama',
 '13': '5. Peter Jackson',
 '14': '3. Robert Rodriguez',
 '15': '3. Chris Hemsworth',
 '16': '3. Matt Damon',
 '17': '3. Action',
 '18': '1. K-19: The Widowmaker (tt0267626)',
 '19': '5. 2015',
 '20': '1. 2014',
 '21': '4. сентябрь',
 '22': '2. 450',
 '23': '5. Peter Jackson',
 '24': '5. Four By Two Productions',
 '25': '3. Midnight Picture Show',
 '26': '1. Inside Out, The Dark Knight, 12 Years a Slave',
 '27': '5. Daniel Radcliffe & Rupert Grint'}

In [65]:
# и убедиться что ни чего не пропустил)
len(answers)

27